In [1]:
import numpy as np
import tensorflow as tf
import edward as ed
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from edward.models import Normal

In [2]:
# make toy data

X, y = make_regression(n_samples=500000, n_features = 20, n_informative=15)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [3]:
N = x_train.shape[0]
D = x_train.shape[1]
M = 50

# set up placeholders
x_ph = tf.placeholder(tf.float32, [None, D])
y_ph = tf.placeholder(tf.float32, [None])

w = Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = Normal(loc=tf.zeros(1.), scale=tf.ones(1.))
y = Normal(loc=ed.dot(x_ph, w) + b, scale=1.0)



In [4]:
qw = Normal(loc=tf.get_variable('qw/loc', [D]),
            scale=tf.nn.softplus(tf.get_variable('qw/scale', [D])))
qb = Normal(loc=tf.get_variable('qb/loc', [1]),
            scale=tf.nn.softplus(tf.get_variable('qb/scale', [1])))

In [5]:
def generator(arrays, batch_size):
  """Generate batches, one with respect to each array's first axis."""
  starts = [0] * len(arrays)  # pointers to where we are in iteration
  while True:
    batches = []
    for i, array in enumerate(arrays):
      start = starts[i]
      stop = start + batch_size
      diff = stop - array.shape[0]
      if diff <= 0:
        batch = array[start:stop]
        starts[i] += batch_size
      else:
        batch = np.concatenate((array[start:], array[:diff]))
        starts[i] = diff
      batches.append(batch)
    yield batches

data = generator([x_train, y_train], M)

In [6]:
n_batch = int(N / M)
n_epoch = 5

inference = ed.KLqp({w: qw, b: qb}, data={y: y_ph})
inference.initialize(
    n_iter=n_batch * n_epoch, n_samples=10, scale={y: N / M}, logdir='log')
tf.global_variables_initializer().run()

In [7]:
for _ in range(inference.n_iter):
    X_batch, y_batch = next(data)
    info_dict = inference.update({x_ph:X_batch, y_ph: y_batch})
    inference.print_progress(info_dict)


40000/40000 [100%] ██████████████████████████████ Elapsed: 112s | Loss: 115042832.000


In [8]:
y_post = ed.copy(y, {w: qw, b: qb})

In [9]:
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={x_ph: x_test, y_post: y_test}))

print("Mean absolute error on test data:")
print(ed.evaluate('mean_absolute_error', data={x_ph: x_test, y_post: y_test}))

Mean squared error on test data:


/Users/allenchang/PycharmProjects/edward_one/venv/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


637.9396
Mean absolute error on test data:
20.15033


In [12]:
!tensorboard --logdir=20181001_161900

TensorBoard 1.6.0 at http://Allens-MacBook-Pro.local:6006 (Press CTRL+C to quit)
^C
